In [66]:
import requests
import lxml.html as lh
import pandas as pd
import numpy as np
import country_converter as coco

### Build df from Url

In [67]:
# root = lh.parse(page.content).getroot()
element = doc.get_element_by_id("nav-today")
print (lh.tostring(element))

b'<div class="tab-pane active" id="nav-today" role="tabpanel" aria-labelledby="nav-today-tab">\n            <div class="main_table_countries_div">\n                <table id="main_table_countries_today" class="table table-bordered table-hover main_table_countries" style="width:100%">\n                    <thead>\n                        <tr>\n                            <th width="100">Country,<br>Other</th>\n                            <th width="20">Total<br>Cases</th>\n                            <th width="30">New<br>Cases</th>\n                            <th width="30">Total<br>Deaths</th>\n                            <th width="30">New<br>Deaths</th>\n                            <th width="30">Total<br>Recovered</th>\n                            <th width="30">Active<br>Cases</th>\n                            <th width="30">Serious,<br>Critical</th>\n                            <th width="30">Tot&#160;Cases/<br>1M pop</th>\n                        </tr>\n                    </th

In [68]:
url='https://www.worldometers.info/coronavirus/'

# Scraping the Url
page = requests.get(url)
doc = lh.fromstring(page.content)
element = doc.get_element_by_id("nav-today")
# print (lh.tostring(element))


# Parse data
th_elements = element.xpath('//th') # header
td_elements = element.xpath('//td') # cells content

headers = [th_element.text_content() for th_element in th_elements]
content = [td_element.text_content() for td_element in td_elements]
rows_content = np.array(content).reshape(int(len(content)/len(headers)),len(headers)).tolist()

df = pd.DataFrame(rows_content)
df.columns = headers
df = df[:-1] # drop Total row

ValueError: cannot reshape array of size 3195 into shape (177,18)

In [65]:
headers

['Country,Other',
 'TotalCases',
 'NewCases',
 'TotalDeaths',
 'NewDeaths',
 'TotalRecovered',
 'ActiveCases',
 'Serious,Critical',
 'Tot\xa0Cases/1M pop',
 'Country,Other',
 'TotalCases',
 'NewCases',
 'TotalDeaths',
 'NewDeaths',
 'TotalRecovered',
 'ActiveCases',
 'Serious,Critical',
 'Tot\xa0Cases/1M pop']

In [52]:
# Convert values to float
for i,col_name in enumerate(df.columns):
    print
    if i!=0:
        df[col_name] = pd.to_numeric(df[col_name].apply(lambda x:x.replace(",","")),errors='coerce')
        
# Convert country to ISO codes
countries_list = df["Country,Other"]\
.apply(lambda x: x.strip()).replace({'UK': 'Great Britain', 'UAE': 'United Arab Emirates'}).values.tolist()
df["iso_alpha"] = pd.Series(coco.convert(names=countries_list, to='ISO3', not_found=None))

# Rename comma seperated cols
df = df.rename(columns={'Country,Other': 'Country',
                  'Serious,Critical': 'Critical'})

df = df.fillna(0)

# Create text that will be display on hover
df["text"] = df['Country'].apply(lambda x: x.strip()) + '<br>' + \
    'Active Cases ' + df['ActiveCases'].astype(int).astype(str) + \
    '<br>' + 'Total Deaths ' + df['TotalDeaths'].astype(int).astype(str)

In [53]:
df.shape

(353, 11)

In [9]:
# Export Dataframe
df.to_csv("static/data/corona.csv",index=False,sep=",")

### Visualize df using Plotly (Optional)

In [ ]:
# import plotly.express as px
# import datetime
# today_date = datetime.datetime.today().date().strftime("%d-%m-%Y")
# fig = px.choropleth(df, locations="iso_alpha",
#                     color="TotalCases",
#                     hover_name="Country",
#                     color_continuous_scale=px.colors.diverging.Portland,
#                    title='Daily Coronavirus Cases in the Word [{}]'.format(today_date)\
#                     +' Source: <a https://www.worldometers.info/coronavirus/">Worldometers</a>',
#                    height=600,
#                    range_color=[0,1000],
#                    labels={'TotalCases':'Min Number of cases'})
# fig.show()